In [1]:
#Import necessary libraries
import os
import ee
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import statsmodels.api as sm
import pandas as pd
from IPython.display import HTML, display
import random
import json
import time
num_seed=30
random.seed(num_seed)

In [2]:
#Initialize earth engine
ee.Initialize()


In [3]:
#Define functions for mapping MapBiomas and simplifying the legend
coverage_palette =  ['ffffff', '129912', '1f4423', '006400', '00ff00', '687537', '76a5af', '29eee4', 
                     '77a605', '935132', 'bbfcac', '45c2a5', 'b8af4f', 'f1c232', 'ffffb2', 'ffd966', 
                     'f6b26b', 'f99f40', 'e974ed', 'd5a6bd', 'c27ba0', 'fff3bf', 'ea9999', 'dd7e6b', 
                     'aa0000', 'ff99ff', '0000ff', 'd5d5e5', 'dd497f', 'b2ae7c', 'af2a2a', '8a2be2', 
                     '968c46', '0000ff', '4fd3ff']


simple_palette = ['129912','BBFCAC','FFFFB2','EA9999','0000FF','D5D5E5']
statesViz = {'min': 0, 'max': 34, 'palette': coverage_palette};
simpleStatesViz = {'min': 1, 'max': 6, 'palette': simple_palette};

change_detection_palette = ['df07b5','0741df']
changeDetectionViz = {'min': 0, 'max': 1, 'palette': change_detection_palette};

#Load in mapbiomas
mapbiomas_states=ee.Image('projects/mapbiomas-workspace/public/collection4_1/mapbiomas_collection41_integration_v1')
states_mask = mapbiomas_states.mask()

#Define function to convert hierarchical legend to simplest form
def simplify_legend(bandName):
    simplify = mapbiomas_states.expression(
        '(b0 >=1)  && (b0<10) ? 1 :'+
        '((b0>=10) && (b0<14)) || (b0==32) || (b0==29) ? 2 :'+
        '((b0>=18) && (b0<22)) || ((b0>=14)&&(b0<16)) ? 3 :'+
        '((b0>=22) && (b0<26)) || (b0==30) ? 4 :'+
        '(b0==26) || (b0==33) || (b0==31) ? 5 : 6', 
        {
          'b0': mapbiomas_states.select([bandName])
        })
    simplify = simplify.select(['constant'],[bandName])
    return simplify

#Select bands we are interested in
bandList = ['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 
             'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 
             'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 
             'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 
             'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 
             'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 
             'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 
             'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 
             'classification_2017', 'classification_2018']
bandsEEList = ee.List(bandList) 
states_simple = ee.ImageCollection(bandsEEList.map(simplify_legend)).toBands()
states_simple = states_simple.select(states_simple.bandNames(),bandsEEList)
states_simple = states_simple.updateMask(states_mask)
states_simple = states_simple.set(ee.Dictionary({'min_value':1,'max_value':5}))
#states_simple is now an iamge where each band corresponds to the land cover class for the band name year

#Map one year to check it out!
Map1 = geemap.Map(center=[-9,-51], zoom=4)
Map1.addLayer(mapbiomas_states.select('classification_2018'),statesViz,name='Original MapBiomas')
Map1.addLayer(states_simple.select('classification_2018'),simpleStatesViz,name='Simplified MapBiomas')
display(Map1)


Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [4]:
#Save 30 meter projection
#Save 30 meter projection
projection_30m = mapbiomas_states.projection().getInfo()
crs = projection_30m.get('crs')
crsTransform = projection_30m.get('transform')
scale = mapbiomas_states.projection().nominalScale().getInfo()
print(crs, crsTransform,scale)


EPSG:4326 [0.0002694945852358564, 0, -74.54381924374933, 0, -0.0002694945852358564, 6.792611020869763] 29.999999999999996


In [5]:
#Convert long band names to short band names
intBandNames = ['1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', 
             '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',  '2005', '2006', '2007', '2008', 
             '2009', '2010', '2011', '2012','2013', '2014', '2015', '2016', '2017', '2018']

states_simple = states_simple.select(bandList,intBandNames)


In [6]:

#Image bands must be ordered by increasing years
def get_year_stack_image_collection(image, band_names, band_indices=[-1,0,1]):
    '''
    Function returns image collection of images where each band is taken from the band_indices. If inputted bands do
                do not follow the band indices, that image will not be returned. 
                For example if one band index is less than 0, an image for the first band will not be returned
                because there is not a band corresponding to that index.
    Inputs:
        image: image where each band represents the land cover classification for a year, bands ordered by 
                increasing years
        band_names: list of band names in the image
        band_indices: list of indices you want to collect from the image, the default [-1,0,1] will return an 
                image collection where each image will have the bands [previous year, current year, following year]
    Returns:
        out_image_list: an image collection where each image corresponds to a band in band_names, where the bands 
                of the image correspond to the band_indices input
                
    Example:
        Inputs:
            image = image of land cover classification for years [1986,1987,1988,1989]
            band_names = [1986,1987,1988,1989]
            band_indices = [-1,0,1]
        Returns:
            out_image_list = image collection with the following images:
                image 1: bands: [1986,1987,1988], property {'OriginalBand': 1987}
                image 2: bands: [1987,1988,1989], property {'OriginalBand': 1988}
            (an image for 1986 is not included because there is not a year before 1986,
             and an image for 1989 is not included because there is not a year after 1989)
    '''
    out_image_list = []
    for i,band_name in enumerate(band_names):
        #indices = i_
        if all(np.array([int(i+x) for x in band_indices])>=0):
            try:
                band_list = [band_names[i+x] for x in band_indices]
                out_image = ee.Image.cat(image.select(band_list))
                out_image = out_image.set(ee.Dictionary({'OriginalBand':band_name}))
                out_image_list.append(out_image)
            except:
                None
    
    return ee.ImageCollection(out_image_list)


In [7]:
#Functions for binary land cover change properties
def lc_one_change(image):
    '''
    Determines if there was one change occurance from year i to year i+1. Returns an image with values:
    1 if state(i) != state(i+1)
    0 if state(i) == state(i+1)
    '''
    band_names = image.bandNames()
    out_image = image.select([band_names.get(0)]).neq(image.select([band_names.get(1)]))
    out_image = out_image.select(out_image.bandNames(),[band_names.get(0)])
    out_image = out_image.set(ee.Dictionary({'OriginalBand':band_names.get(0)}))
    return out_image

def lc_consistent_change_one_year(image):
    '''
    Determines if change that occured from i to i+1 stayed in i+2
    1 if state(i) != state(i+1) and state(i+1) == state(i+2)
    0 otherwise
    '''
    band_names = image.bandNames()
    current_year = image.select([band_names.get(0)])
    next_year = image.select([band_names.get(1)])
    next_next_year = image.select([band_names.get(2)])
    
    changed = current_year.neq(next_year)
    stayed = next_year.eq(next_next_year)
    
    out_image = changed.bitwise_and(stayed)
    out_image = out_image.select(out_image.bandNames(),[band_names.get(0)])
    out_image = out_image.set(ee.Dictionary({'OriginalBand':band_names.get(0)}))
    return out_image

def lc_consistent_change_two_years(image):
    '''
    Determines if change that occured from i to i+1 stayed in i+2 and i+3
    1 if state(i) != state(i+1) and state(i+1) == state(i+2) and state(i+1) == state(i+3)
    0 otherwise
    '''
    band_names = image.bandNames()
    current_year = image.select([band_names.get(0)])
    next_year = image.select([band_names.get(1)])
    next_next_year = image.select([band_names.get(2)])
    next_next_next_year = image.select([band_names.get(3)])
    
    changed = current_year.neq(next_year)
    stayed = next_year.eq(next_next_year)
    stayed_again = next_year.eq(next_next_next_year)
    
    out_image = changed.bitwise_and(stayed.bitwise_and(stayed_again))
    out_image = out_image.select(out_image.bandNames(),[band_names.get(0)])
    out_image = out_image.set(ee.Dictionary({'OriginalBand':band_names.get(0)}))
    return out_image


In [12]:
#Apply land cover change functions to images, first returns an image collection then converted to image
#where each band represents one year
lc_one_change_col = get_year_stack_image_collection(states_simple,states_simple.bandNames().getInfo(), band_indices=[0,1])
lc_one_change_col = lc_one_change_col.map(lc_one_change)
lc_one_change_image = lc_one_change_col.toBands()
lc_one_change_image = lc_one_change_image.select(lc_one_change_image.bandNames(),lc_one_change_col.aggregate_array('OriginalBand'))

lc_consistent_change_two_years_col = get_year_stack_image_collection(states_simple,states_simple.bandNames().getInfo(), band_indices=[0,1,2,3])
lc_consistent_change_two_years_col = lc_consistent_change_two_years_col.map(lc_consistent_change_two_years)
lc_consistent_change_two_years_image = lc_consistent_change_two_years_col.toBands()
lc_consistent_change_two_years_image = lc_consistent_change_two_years_image.select(lc_consistent_change_two_years_image.bandNames(), lc_consistent_change_two_years_col.aggregate_array('OriginalBand'))

#lc_one_change_image = binary image representing if change occured in that pixel over the entire period
#lc_consistent_change_two_years_image = binary image representing if consistent change occured in that pixel over the entire period



In [13]:
#Mapchange layer for one year to test if it works
Map2 = geemap.Map(center=[-9,-51], zoom=4)
Map2.addLayer(lc_one_change_image.select(['1990']),changeDetectionViz,name='1990 One Change')
Map2.addLayer(states_simple.select(['1990']),simpleStatesViz,name='1990')
Map2.addLayer(states_simple.select(['1991']),simpleStatesViz,name='1991')
display(Map2)


Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [14]:
#Map consistent change layer for one year to test if it works
Map2 = geemap.Map(center=[-9,-51], zoom=4)
Map2.addLayer(lc_consistent_change_two_years_image.select(['1990']),changeDetectionViz,name='1990 Consistent Change')
Map2.addLayer(states_simple.select(['1990']),simpleStatesViz,name='1990')
Map2.addLayer(states_simple.select(['1991']),simpleStatesViz,name='1991')
Map2.addLayer(states_simple.select(['1992']),simpleStatesViz,name='1992')
display(Map2)

Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [15]:
#Mask Change images
lc_one_change_image_masked = lc_one_change_image.selfMask()
lc_consistent_change_two_years_image_masked = lc_consistent_change_two_years_image.selfMask()


In [17]:
#Build current state and year after state collection, one image for each year
#First band represents the current year's state, second band represents the year after's state
transitions_collection = get_year_stack_image_collection(states_simple,states_simple.bandNames().getInfo(), band_indices=[0,1])
#Has property "OriginalBand" for the year of the current state (band 0)
#Right now has default transform crs_transform': [1, 0, 0, 0, 1, 0], so in sampling the image you have to 
#specify the projection adn resolution


{'type': 'Image', 'bands': [{'id': '1985', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 6}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': '1986', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 6}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'min_value': 1, 'OriginalBand': '1985', 'max_value': 5, 'system:index': '0'}}


In [18]:
#Function to sample image data at point locations (sampleBandPoints) and rename new property to image_name
def getSampleImageData(image, sampleBandPoints, image_name, crs, crsTransform):
    #Sample image data at point locations
    sampleImageData = image.reduceRegions(
        collection=sampleBandPoints,
        reducer=ee.Reducer.first(),
        crs=crs,
        crsTransform=crsTransform
        )
    #Rename sampled values from "first" to image_name
    sampleImageData = sampleImageData.map(lambda x: x.set({image_name:x.get('first')}))
    return sampleImageData



In [ ]:
#Code below to export feature collection to drive, if you get an error that the task is too large,
#try to break up the feature collection into chunks

# export_sample_points_task = ee.batch.Export.table.toDrive(
#     collection=out_feature_collection, 
#     description = "Training_Set_with_Covariates_10K_{}".format(i), 
#     fileNamePrefix = 'Training_Set_with_Covariates_10K_{}'.format(i),
#     folder = 'Dynamic World')

#     export_sample_points_task.start()
#     print(export_sample_points_task)